In [ ]:
import gc
import numpy as np
import pandas as pd

import sys
import os
sys.path.append(os.path.dirname(sys.path[0]))

from loguru import logger

from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.linear_model import QuantileRegressor, LinearRegression
from sklearn.metrics import mean_pinball_loss, mean_squared_error
from sklearn.model_selection import cross_validate, TimeSeriesSplit
from tqdm import tqdm
from sklearn.inspection import permutation_importance
import scikit_posthocs as sp
import matplotlib.pyplot as plt

from sklearn.utils.fixes import parse_version, sp_version
solver = "highs" if sp_version >= parse_version("1.6.0") else "interior-point"

In [ ]:
# logger_format = (
#     "<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green> | "
#     "<level>{level: <8}</level> | "
#     "<cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> | "
#     "{extra[ip]} {extra[user]} - <level>{message}</level>"
# )
# logger.configure(extra={"ip": "", "user": ""})  # Default values
# logger.remove()
# logger.add(sys.stderr, format=logger_format)

# logger.add(sys.stderr, format=logger_format, level="TRACE")

# from loguru import logger
# logger.add(
#     'info.log',
#     format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {module}:{function}:{line} - {message}",
#     level="INFO",
# )
# def main():
#     logger.debug("This is a debug message")
#     logger.info("This is an info message")
#     logger.warning("This is a warning message")
#     logger.error("This is an error message")
# if __name__ == "__main__":
#     main()

In [ ]:
########################### UTILS FILE 
def filter_offshore(df, offshore_filter):
    df = df[df.offshoreonshore == offshore_filter]  # filter by offshore/onshore
    return df

def set_index_datetiemUTC(df):
    df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
    df = df.sort_values(by='datetime').reset_index(drop=True)
    df.set_index('datetime', inplace=True)
    return df

def process_file(file, offshore_filter='Offshore'):
    df = pd.read_json(file)
    df = filter_offshore(df, offshore_filter)
    df = set_index_datetiemUTC(df)
    df[df['measured'] < 0] = 0
    return df

########################### UTILS DATA PROCESSING   
def scale(df, col_name, max_cap):
    df_ = df.copy()
    values = df_[col_name].values
    return values/max_cap

def detect_ramp_event(df, ramp_threshold):
    df['ramp_event'] = (np.abs(df['diff_norm_measured']) > ramp_threshold).astype(int)
    df['ramp_event_up'] = (df['diff_norm_measured'] > ramp_threshold).astype(int)
    df['ramp_event_down'] = (df['diff_norm_measured'] < -ramp_threshold).astype(int)
    return df

############################# UTILS PREDICTIONS
def dict2df_predictions(prediction, col_name):
    df_pred = pd.DataFrame.from_records(prediction)
    df_pred.set_index('datetime', inplace=True)
    df_pred.columns = [col_name + '_pred']
    return df_pred

def dict2df_quantiles10(prediction, col_name):
    df_pred = pd.DataFrame.from_records(prediction)
    df_pred.set_index('datetime', inplace=True)
    df_pred.columns = [col_name + '_quantile10']
    return df_pred

def dict2df_quantiles90(prediction, col_name):
    df_pred = pd.DataFrame.from_records(prediction)
    df_pred.set_index('datetime', inplace=True)
    df_pred.columns = [col_name + '_quantile90']
    return df_pred

# Timestamp training and prediction

In [ ]:
def generate_timestamps(start_training, i, window_size):
    # Generate timestamps for training and prediction
    assert window_size > 0, "Window size must be greater than 0"
    start_training_timestamp = pd.to_datetime(start_training, utc=True) + pd.Timedelta(days=i)
    end_training_timestamp = pd.to_datetime(start_training, utc=True) + pd.Timedelta(days=i + window_size)
    start_prediction_timestamp = pd.to_datetime(start_training, utc=True) + pd.Timedelta(days=i + window_size)
    end_prediction_timestamp = pd.to_datetime(start_training, utc=True) + pd.Timedelta(days=i + window_size + 1)
    return start_training_timestamp, end_training_timestamp, start_prediction_timestamp, end_prediction_timestamp

# Forecasters

## ELIA

### Deterministic

In [ ]:
# day ahead
def create_day_ahead_predictions(df_val):
    " Create day ahead predictions "
    day_ahead_elia_prediction = []
    for i in range(len(df_val)):
        day_ahead_elia_prediction.append({'datetime': df_val.index[i],
                                            'predictions': df_val.dayaheadforecast.iloc[i]})
    df_day_ahead_pred = dict2df_predictions(day_ahead_elia_prediction, 'day_ahead')
    return df_day_ahead_pred

# day ahead 11
def create_day_ahead_11_predictions(df_val):
    " Create day ahead 11 predictions"
    day_ahead_11_elia_prediction = []
    for i in range(len(df_val)):
        day_ahead_11_elia_prediction.append({'datetime': df_val.index[i],
                                                'predictions': df_val.dayahead11hforecast.iloc[i]})
    df_day_ahead11_pred = dict2df_predictions(day_ahead_11_elia_prediction, 'day_ahead11')
    return df_day_ahead11_pred

# week ahead
def create_week_ahead_predictions(df_val):
    " Create week ahead predictions"
    week_ahead_elia_prediction = []
    for i in range(len(df_val)):
        week_ahead_elia_prediction.append({'datetime': df_val.index[i],
                                            'predictions': df_val.weekaheadforecast.iloc[i]})
    df_week_ahead_pred = dict2df_predictions(week_ahead_elia_prediction, 'week_ahead')
    return df_week_ahead_pred

### probabilistic

In [ ]:
# day ahead
def create_day_ahead_quantiles10(df_val):
    " Create day ahead quantiles 10"
    day_ahead_elia_quantile10 = []
    for i in range(len(df_val)):
        day_ahead_elia_quantile10.append({'datetime': df_val.index[i],
                                            'quantiles10': df_val.dayaheadconfidence10.iloc[i]})
    df_day_ahead_quantile10 = dict2df_quantiles10(day_ahead_elia_quantile10, 'day_ahead')
    return df_day_ahead_quantile10

# day ahead 11
def create_day_ahead_11_quantiles10(df_val):
    " Create day ahead 11 quantiles 10"
    day_ahead_11_elia_quantile10 = []
    for i in range(len(df_val)):
        day_ahead_11_elia_quantile10.append({'datetime': df_val.index[i],
                                                'quantiles10': df_val.dayahead11hconfidence10.iloc[i]})
    df_day_ahead11_quantile10 = dict2df_quantiles10(day_ahead_11_elia_quantile10, 'day_ahead11')
    return df_day_ahead11_quantile10

# week ahead
def create_week_ahead_quantiles10(df_val):
    " Create week ahead quantiles 10"
    week_ahead_elia_quantile10 = []
    for i in range(len(df_val)):
        week_ahead_elia_quantile10.append({'datetime': df_val.index[i],
                                            'quantiles10': df_val.weekaheadconfidence10.iloc[i]})
    df_week_ahead_quantile10 = dict2df_quantiles10(week_ahead_elia_quantile10, 'week_ahead')
    return df_week_ahead_quantile10

# --------------------------------

# day ahead
def create_day_ahead_quantiles90(df_val):
    " Create day ahead quantiles 90"
    day_ahead_elia_quantile90 = []
    for i in range(len(df_val)):
        day_ahead_elia_quantile90.append({'datetime': df_val.index[i],
                                            'quantiles90': df_val.dayaheadconfidence90.iloc[i]})
    df_day_ahead_quantile90 = dict2df_quantiles90(day_ahead_elia_quantile90, 'day_ahead')
    return df_day_ahead_quantile90

# day ahead 11
def create_day_ahead_11_quantiles90(df_val):
    " Create day ahead 11 quantiles 90"
    day_ahead_11_elia_quantile90 = []
    for i in range(len(df_val)):
        day_ahead_11_elia_quantile90.append({'datetime': df_val.index[i],
                                                'quantiles90': df_val.dayahead11hconfidence90.iloc[i]})
    df_day_ahead11_quantile90 = dict2df_quantiles90(day_ahead_11_elia_quantile90, 'day_ahead11')
    return df_day_ahead11_quantile90

# week ahead
def create_week_ahead_quantiles90(df_val):
    " Create week ahead quantiles 90"
    week_ahead_elia_quantile90 = []
    for i in range(len(df_val)):
        week_ahead_elia_quantile90.append({'datetime': df_val.index[i],
                                            'quantiles90': df_val.weekaheadconfidence90.iloc[i]})
    df_week_ahead_quantile90 = dict2df_quantiles90(week_ahead_elia_quantile90, 'week_ahead')
    return df_week_ahead_quantile90

# Ensemble Learning

##  Stacked Generalization (Quantile Regressor/ Quantile GB Regressor)

## feature engineering

In [ ]:
def normalize_dataframe(df, maximum_capacity):
    " Normalize dataframe by dividing by maximum capacity"
    assert maximum_capacity > 0, "Maximum capacity must be greater than 0"
    df_normalized = df.copy()
    for col in df_normalized.columns:
        normalize_col = scale(df_normalized, col, maximum_capacity)
        df_normalized[f'norm_{col}'] = normalize_col
    return df_normalized.filter(like='norm')

def differentiate_dataframe(df):
    " Differentiate dataframe by computing the absolute difference between consecutive values"
    df_differential = df.copy()
    for col in df_differential.columns:
        df_differential[f'diff_{col}'] = df_differential[col].diff()
    return df_differential.filter(like='diff').iloc[1:]

def create_augmented_dataframe(df, max_lags, forecasters_diversity=True, lagged=True, augmented=True, differenciate=True):
    " Create feature engineering dataframe with forecasters diversity, lagged, augmented and differenciate features"
    shifted_df_ensemble = pd.DataFrame()

    if lagged:
        " Create lagged features"
        for lag in range(1, max_lags + 1):
            for col in df.columns:
                shifted_df_ensemble[col+'_t-'+str(lag)] = df[col].shift(lag)  # lagged
    if augmented:
        " Create augmented features"
        for col in df.columns:
            shifted_df_ensemble[col + "_sqr"] = df[col]**2  # squared
            shifted_df_ensemble[col + "_std"] = df[col].rolling(max_lags).std()  # rolling standard deviation
            shifted_df_ensemble[col + "_var"] = df[col].rolling(max_lags).var()  # rolling variance
            if max_lags > 2:
                shifted_df_ensemble[col + "_lag-1_std"] = df[col].shift(1).rolling(max_lags-1).std()  # rolling standard deviation on lag-1
                shifted_df_ensemble[col + "_lag-1_var"] = df[col].shift(1).rolling(max_lags-1).var()  # rolling variance on lag-1
    if differenciate:
        " Create differenciate features"
        for col in df.columns:
            shifted_df_ensemble[col + "_diff"] = df[col].diff()  # difference
            shifted_df_ensemble[col + "_lag-1_diff"] = df[col].shift(1).diff()  # difference on lag-1
    if forecasters_diversity:
        " Create forecasters diversity features"
        forecast_cols = [name for name in df.columns if 'pred' in name]  # forecasters columns
        shifted_df_ensemble["forecasters_std"] = df[forecast_cols].std(axis=1)  # standard deviation among forecasters
        shifted_df_ensemble["forecasters_var"] = df[forecast_cols].var(axis=1)  # variance among forecasters
    " Concatenate the original dataframe with the shifted dataframe"
    df = pd.concat([df, shifted_df_ensemble], axis=1)
    df = df.iloc[max_lags:,:]
    return df

def prepare_train_test_data(df_ensemble, df_val, df_test, start_predictions):
    df_train_ensemble = df_ensemble[df_ensemble.index < start_predictions].copy()
    df_train_ensemble.loc[:, 'diff_norm_targ'] = df_val['diff_norm_measured']
    df_test_ensemble = df_ensemble[df_ensemble.index >= start_predictions].copy()
    df_test_ensemble.loc[:, 'diff_norm_targ'] = df_test['diff_norm_measured']
    return df_train_ensemble, df_test_ensemble

def get_numpy_Xy_train_test(df_train_ensemble, df_test_ensemble):
    X_train, y_train = df_train_ensemble.iloc[:, :-1].values, df_train_ensemble.iloc[:, -1].values
    X_test, y_test = df_test_ensemble.iloc[:, :-1].values, df_test_ensemble.iloc[:, -1].values
    return X_train, y_train, X_test, y_test

def augment_with_quantiles(X_train, X_test, df_train_ensemble, 
                            X_train_quantile10, X_test_quantile10, df_train_ensemble_quantile10,
                            X_train_quantile90, X_test_quantile90, df_train_ensemble_quantile90,  
                            quantile, augment_q50=False):
    " Augment the training and testing data with the quantiles predictions"
    quantile_data = {
        0.1: (X_train_quantile10, X_test_quantile10, df_train_ensemble_quantile10),
        0.5: (np.concatenate([X_train_quantile10, X_train_quantile90], axis=1),
                np.concatenate([X_test_quantile10, X_test_quantile90], axis=1),
                pd.concat([df_train_ensemble_quantile10, df_train_ensemble_quantile90], axis=1)),
        0.9: (X_train_quantile90, X_test_quantile90, df_train_ensemble_quantile90)
    }
    if quantile not in quantile_data:
        raise ValueError('Invalid quantile value. Must be 0.1, 0.5, or 0.9.')
    " Get the quantile data and augment the training and testing data with it"
    X_train_part, X_test_part, df_train_ensemble_part = quantile_data[quantile]
    if quantile == 0.5 and not augment_q50:
        " Do not augment with augmented quantiles"
        return X_train, X_test, df_train_ensemble
    X_train = np.concatenate([X_train, X_train_part], axis=1)
    X_test = np.concatenate([X_test, X_test_part], axis=1)
    df_train_ensemble = pd.concat([df_train_ensemble, df_train_ensemble_part], axis=1)
    return X_train, X_test, df_train_ensemble


## Hyperparms QR Optimization

In [ ]:
" Evaluate model using Pnball loss."
def score_func_10(estimator, X, y):
    y_pred = estimator.predict(X)
    return {
        "mean_pinball_loss": mean_pinball_loss(y, y_pred, alpha=0.1),
    }
def score_func_50(estimator, X, y):
    y_pred = estimator.predict(X)
    return {
        "mean_pinball_loss": mean_squared_error(y, y_pred), # mean_pinball_loss(y, y_pred, alpha=0.5),
    }
def score_func_90(estimator, X, y):
    y_pred = estimator.predict(X)
    return {
        "mean_pinball_loss": mean_pinball_loss(y, y_pred, alpha=0.9),
    }

score_func = {0.1: score_func_10,
                0.5: score_func_50,
                0.9: score_func_90}

def evaluate(model, X, y, cv, quantile, score_func):
    cv_results = cross_validate(
        model,
        X,
        y,
        cv=cv,
        scoring=score_func[quantile],
        n_jobs=6
    )
    score_mean = cv_results['test_mean_pinball_loss'].mean()
    return score_mean

def optimize_gbr(X_train, y_train, quantile, nr_cv_splits, config_params, score_func):
    " Hyperparameter optimization for Quantile Gradient Boosting Regressor."
    best_gbr_params = None
    best_score=100000
    ts_cv = TimeSeriesSplit(n_splits=nr_cv_splits)
    for learning_rate in config_params['learning_rate']:
        for subsample in config_params['max_features']:
            for max_depth in config_params['max_depth']:
                for n_estimators in config_params['max_iter']:
                    gbr_params = dict(
                        learning_rate=learning_rate,
                        max_features = subsample,
                        max_iter=n_estimators,
                        max_depth=max_depth,
                        random_state=42)
                    if quantile == 0.5:
                        gbr = HistGradientBoostingRegressor(**gbr_params)
                    else:
                        gbr = HistGradientBoostingRegressor(loss="quantile", quantile=quantile, **gbr_params)
                    mean_cv_score = evaluate(gbr, X_train, y_train, cv=ts_cv, quantile=quantile, score_func=score_func) 
                    if mean_cv_score < best_score:
                        best_score = mean_cv_score
                        best_gbr_params = gbr_params
    return best_score, best_gbr_params

def optimize_lr(X_train, y_train, quantile, nr_cv_splits, solver, config_params, score_func):
    " Hyperparameter optimization for Quantile Linear Regression. "
    best_lr_params = None
    best_score=100000
    ts_cv = TimeSeriesSplit(n_splits=nr_cv_splits)
    for alpha in config_params['alpha']:
        for fit_intercept in config_params['fit_intercept']:
            if quantile == 0.5:
                lr_params = dict(
                fit_intercept=fit_intercept)
                lr = LinearRegression(**lr_params)
            else:
                lr_params = dict(
                alpha=alpha,
                fit_intercept=fit_intercept)
                lr = QuantileRegressor(quantile=quantile, solver=solver, **lr_params)
            mean_cv_score = evaluate(lr, X_train, y_train, cv=ts_cv, quantile=quantile, score_func=score_func)  
            if mean_cv_score < best_score:
                best_score = mean_cv_score
                best_lr_params = lr_params
    return best_score, best_lr_params

def optimize_model(X_train, y_train, quantile, nr_cv_splits, model_type, solver, gbr_config_params, lr_config_params, score_func, logger):
    " Optimize selected model hyperparameters."
    if model_type == 'GBR':
        best_score, best_params = optimize_gbr(X_train, y_train, quantile, nr_cv_splits, gbr_config_params, score_func)
    elif model_type == 'LR':
        best_score, best_params = optimize_lr(X_train, y_train, quantile, nr_cv_splits, solver, lr_config_params, score_func)
    else:
        raise ValueError('"model_type" is not valid')
    logger.info(f'best_score {round(best_score, 3)}')
    logger.info(f'best_params {best_params}')
    return best_score, best_params

def initialize_model(model_type, quantile, best_params):
    " Initialize selected model."
    assert best_params is not None, "Best parameters must be provided"
    if model_type == 'GBR':
        if quantile == 0.5:
            model = HistGradientBoostingRegressor(**best_params)
        else:
            model = HistGradientBoostingRegressor(loss="quantile", quantile=quantile, **best_params)
    elif model_type == 'LR':
        if quantile == 0.5:
            model = LinearRegression(**best_params)
        else:
            model = QuantileRegressor(quantile=quantile, solver=solver, **best_params)
    else:
        raise ValueError('"model_type" is not valid')
    return model

## collect results

In [ ]:
def collect_quantile_ensemble_predictions(quantiles, test_data, predictions):
    " Collect quantile ensemble predictions as a list of dictionaries."
    assert test_data.shape[0] == len(predictions[quantiles[0]]), "Length mismatch between test data and predictions"    
    quantile_predictions_dict = {}
    try:
        for quantile in quantiles:
            quantile_ensemble_predictions = []
            if test_data.shape[0] != len(predictions[quantile]):
                raise ValueError("Length mismatch between test data and predictions for quantile {}".format(quantile))
            for i in range(len(predictions[quantile])):
                quantile_ensemble_predictions.append({'datetime': test_data.index[i],
                                                        'predictions': predictions[quantile][i]})
            quantile_predictions_dict[quantile] = quantile_ensemble_predictions
    except Exception as e:
        logger.exception("An error occurred:", e)
        return None
    return quantile_predictions_dict

def create_ensemble_dataframe(quantiles, quantile_predictions_dict, df_test):
    " Create ensemble dataframe from quantile predictions."
    assert len(quantiles) == len(quantile_predictions_dict), "Length mismatch between quantiles and quantile predictions"
    assert df_test.shape[0] == len(quantile_predictions_dict[quantiles[0]]), "Length mismatch between test data and predictions"
    assert 'diff_norm_measured' in df_test.columns, 'diff_norm_measured column not found in test data'
    for i, quantile in enumerate(quantiles):
        if i == 0:
            df_pred_ensemble = pd.DataFrame(quantile_predictions_dict[quantile])
            df_pred_ensemble.columns = ['datetime', str(int(quantile*100))+'_predictions']
            df_pred_ensemble.set_index('datetime', inplace=True)
        else:
            df_pred_quantile = pd.DataFrame(quantile_predictions_dict[quantile])
            df_pred_quantile.columns = ['datetime', str(int(quantile*100))+'_predictions']
            df_pred_quantile.set_index('datetime', inplace=True)
            df_pred_ensemble = pd.concat([df_pred_ensemble, df_pred_quantile], axis=1)
    df_pred_ensemble['diff_norm_measured'] = df_test['diff_norm_measured']
    return df_pred_ensemble


## 2-stage dataframe

In [ ]:
def create_augmented_dataframe_2stage(df_2stage, order_diff, max_lags, augment=False):
    " Process 2-stage ensemble dataframe with lags."
    assert order_diff > 0, "Order of differentiation must be greater than 0"
    assert max_lags > 0, "Maximum number of lags must be greater than 0"
    # Differentiate the dataframe
    df_2stage_diff = df_2stage.diff(order_diff)
    # Create lagged features
    for lag in range(1, max_lags + 1):
        df_2stage_diff[f'predictions_t-{lag}'] = df_2stage_diff['predictions'].shift(lag)
    if augment:
        for col in df_2stage_diff.columns:
            if 'targets' not in col:
                df_2stage_diff[f'{col}_sqr'] = df_2stage_diff[col]**2
    # Drop rows with NaNs resulting from the shift operation
    df_2stage_process = df_2stage_diff.iloc[max_lags:, :].dropna()
    return df_2stage_process

def create_2stage_dataframe(df_train_ensemble, df_test_ensemble, y_train, y_test, predictions_insample, predictions_outsample):
    " Create 2-stage ensemble dataframe."
    # Creating DataFrame for in-sample predictions
    df_insample = pd.DataFrame(predictions_insample, columns=['predictions'], index=df_train_ensemble.index)
    df_insample['targets'] = y_train
    # Creating DataFrame for out-sample predictions
    df_outsample = pd.DataFrame(predictions_outsample, columns=['predictions'], index=df_test_ensemble.index)
    df_outsample['targets'] = y_test
    # Concatenating in-sample and out-sample DataFrames
    df_2stage = pd.concat([df_insample, df_outsample], axis=0)
    return df_2stage

def create_var_ensemble_dataframe(quantiles, quantile_predictions_dict, df_test):
    " Create ensemble dataframe from quantile predictions."
    assert len(quantiles) == len(quantile_predictions_dict), "Length mismatch between quantiles and quantile predictions"
    assert df_test.shape[0] == len(quantile_predictions_dict[quantiles[0]]), "Length mismatch between test data and predictions"
    for i, quantile in enumerate(quantiles):
        if i == 0:
            df_pred_ensemble = pd.DataFrame(quantile_predictions_dict[quantile])
            df_pred_ensemble.columns = ['datetime', str(int(quantile*100))+'_var_predictions']
            df_pred_ensemble.set_index('datetime', inplace=True)
        else:
            df_pred_quantile = pd.DataFrame(quantile_predictions_dict[quantile])
            df_pred_quantile.columns = ['datetime', str(int(quantile*100))+'_var_predictions']
            df_pred_quantile.set_index('datetime', inplace=True)
            df_pred_ensemble = pd.concat([df_pred_ensemble, df_pred_quantile], axis=1)
    df_pred_ensemble['targets'] = df_test['targets']
    return df_pred_ensemble

## metrics

In [ ]:
def rmse(y_pred, y_targ):
    " Compute Root Mean Squared Error."
    assert len(y_pred) == len(y_targ), "Length mismatch between predictions and targets"
    return np.sqrt(np.mean((y_pred - y_targ)**2))

def calculate_rmse(df, pred_col, targ_col='diff_norm_measured'):
    " Calculate RMSE Loss."
    assert pred_col in df.columns, "prediction column is missing"
    assert targ_col in df.columns, "target column is missing"
    rmse_loss = pd.DataFrame()
    rmse_loss['rmse'] = np.array([rmse(df[pred_col], df[targ_col])])
    return rmse_loss

def calculate_pinball_losses(df, confidence_10_col, confidence_90_col, targ_col='diff_norm_measured'):
    " Calculate Pinball Losses for 10% and 90% quantiles."
    assert 'diff_norm_measured' in df.columns, "diff_norm_measured column is missing"
    pinball_losses = pd.DataFrame()
    score_10 = mean_pinball_loss( list(df[targ_col].values),  list(df[confidence_10_col].values), alpha=0.1)
    score_90 = mean_pinball_loss( list(df[targ_col].values),  list(df[confidence_90_col].values), alpha=0.9)
    pinball_losses['pb_loss_10'] =  np.array([score_10]) 
    pinball_losses['pb_loss_90'] = np.array([score_90]) 
    return pinball_losses

## scheme equal weights ensemble

In [ ]:
def calculate_equal_weights(df_test_norm_diff):
    " Calculate the mean prediction and quantiles using equal weights"
    assert 'diff_norm_measured' in df_test_norm_diff.columns, "diff_norm_measured column is missing"
    Q10 = df_test_norm_diff[['diff_norm_weekaheadconfidence10', 
                                'diff_norm_dayaheadconfidence10', 
                                'diff_norm_dayahead11hconfidence10']].mean(axis=1)
    MEAN = df_test_norm_diff[['diff_norm_weekaheadforecast', 
                                'diff_norm_dayaheadforecast', 
                                'diff_norm_dayahead11hforecast']].mean(axis=1)
    Q90 = df_test_norm_diff[['diff_norm_weekaheadconfidence90', 
                                'diff_norm_dayaheadconfidence90', 
                                'diff_norm_dayahead11hconfidence90']].mean(axis=1)
    df_equal_weights = pd.DataFrame({
        'Q10': Q10,
        'mean_prediction': MEAN,
        'Q90': Q90
    }, index=df_test_norm_diff.index)
    df_equal_weights['diff_norm_measured'] = df_test_norm_diff['diff_norm_measured']
    return df_equal_weights

## weighted average ensemble

In [ ]:
def calculate_weights(df_val_norm_diff):
    " Calculate weights based on the pinball loss of the forecasts"
    assert len(df_val_norm_diff) > 0, 'Dataframe is empty'
    lst_cols = [name for name in list(df_val_norm_diff.columns) if 'mostrecent' not in name]
    targ_col = [name for name in lst_cols if 'measured' in name]
    targets =  df_val_norm_diff[targ_col[0]]
    lst_cols_forecasts = [name for name in lst_cols if 'measured' not in name]
    lst_q10_weight = []
    lst_q50_weight = []
    lst_q90_weight = []
    for col in lst_cols_forecasts:
        if 'forecast' in col:
            forecast = df_val_norm_diff[col]
            q50_pb_loss = mean_squared_error(targets.values,  forecast.values)  # mean_pinball_loss(targets.values,  forecast.values, alpha=0.5)
            lst_q50_weight.append({col : 1/q50_pb_loss})
        elif 'confidence10' in col:
            forecast = df_val_norm_diff[col]
            q10_pb_loss = mean_pinball_loss(targets.values,  forecast.values, alpha=0.1)
            lst_q10_weight.append({col : 1/q10_pb_loss})
        elif 'confidence90' in col:
            forecast = df_val_norm_diff[col]
            q90_pb_loss = mean_pinball_loss(targets.values,  forecast.values, alpha=0.9)
            lst_q90_weight.append({col : 1/q90_pb_loss})
        else:
            raise ValueError('Not a valid column')
    return lst_cols_forecasts, lst_q10_weight, lst_q50_weight, lst_q90_weight
    
def normalize_weights(lst_weight):
    " Normalize weights, the sum should be 1"
    assert len(lst_weight) > 0, 'List of weights is empty'  
    total_sum = sum(list(loss.values())[0] for loss in lst_weight)
    norm_lst_weight = [{key: value/total_sum for key, value in d.items()} for d in lst_weight]
    return norm_lst_weight

def calculate_combination_forecast(df_test_norm_diff, lst_cols_forecasts, norm_lst_q50_pb_loss, norm_lst_q10_pb_loss, norm_lst_q90_pb_loss):
    " Calculate the combination forecast based on the pinball loss-based weights"
    combination_forecast = np.zeros(len(df_test_norm_diff))
    combination_quantile10 = np.zeros(len(df_test_norm_diff))
    combination_quantile90 = np.zeros(len(df_test_norm_diff))
    for col in lst_cols_forecasts:
        if 'forecast' in col:
            forecast = df_test_norm_diff[col]
            weight = [list(weight.values())[0] for weight in norm_lst_q50_pb_loss if col in weight][0]
            combination_forecast += forecast * weight
        elif 'confidence10' in col:
            forecast = df_test_norm_diff[col]
            weight = [list(weight.values())[0] for weight in norm_lst_q10_pb_loss if col in weight][0]
            combination_quantile10 += forecast * weight
        elif 'confidence90' in col:
            forecast = df_test_norm_diff[col]
            weight = [list(weight.values())[0] for weight in norm_lst_q90_pb_loss if col in weight][0]
            combination_quantile90 += forecast * weight
        else:
            raise ValueError('Not a valid column')
    return combination_forecast, combination_quantile10, combination_quantile90

def create_weighted_avg_df(df_test_norm_diff, combination_forecast, combination_quantile10, combination_quantile90):
    " Create dataframe with the weighted average forecast"
    assert len(df_test_norm_diff) == len(combination_forecast) == len(combination_quantile10) == len(combination_quantile90), 'Length mismatch'
    df_weighted_avg = pd.DataFrame({
        'Q10': combination_quantile10,
        'mean_prediction': combination_forecast,
        'Q90': combination_quantile90
    }, index=df_test_norm_diff.index)
    df_weighted_avg['diff_norm_measured'] = df_test_norm_diff['diff_norm_measured']
    return df_weighted_avg

def calculate_weighted_avg(df_train_norm_diff, df_test_norm_diff, start_predictions, window_size_valid=1, var=False):
    " Calculate the weights based on the pinball loss of the forecasts "
    if var:
        df_diff = pd.concat([df_train_norm_diff, df_test_norm_diff], axis=0).diff().dropna()
        df_train_norm_diff, df_test_norm_diff = df_diff[df_diff.index < start_predictions], df_diff[df_diff.index >= start_predictions]
        window_validation =  pd.to_datetime(start_predictions, utc=True) - pd.Timedelta(days=window_size_valid)
        df_val_norm_diff = df_train_norm_diff[df_train_norm_diff.index.to_series().between(window_validation, start_predictions)]
        lst_cols_forecasts, lst_q10_weight, lst_q50_weight, lst_q90_weight = calculate_weights(df_val_norm_diff)
        norm_lst_q50_weight = normalize_weights(lst_q50_weight) 
        norm_lst_q10_weight = normalize_weights(lst_q10_weight) 
        norm_lst_q90_weight = normalize_weights(lst_q90_weight) 
        combination_forecast, _, _ = calculate_combination_forecast(df_test_norm_diff, lst_cols_forecasts, norm_lst_q50_weight, norm_lst_q10_weight, norm_lst_q90_weight)
        df_weighted_avg = pd.DataFrame({
                'mean_prediction': combination_forecast,
            }, index=df_test_norm_diff.index)
        df_weighted_avg['diff_norm_measured'] = df_test_norm_diff['diff_norm_measured']
        return df_weighted_avg
    window_validation =  pd.to_datetime(start_predictions, utc=True) - pd.Timedelta(days=window_size_valid)
    df_val_norm_diff = df_train_norm_diff[df_train_norm_diff.index.to_series().between(window_validation, start_predictions)]
    lst_cols_forecasts, lst_q10_weight, lst_q50_weight, lst_q90_weight = calculate_weights(df_val_norm_diff)
    norm_lst_q50_weight = normalize_weights(lst_q50_weight) 
    norm_lst_q10_weight = normalize_weights(lst_q10_weight) 
    norm_lst_q90_weight = normalize_weights(lst_q90_weight) 
    combination_forecast, combination_quantile10, combination_quantile90 = calculate_combination_forecast(df_test_norm_diff, lst_cols_forecasts, norm_lst_q50_weight, norm_lst_q10_weight, norm_lst_q90_weight)
    df_weighted_avg = pd.DataFrame({
            'Q10': combination_quantile10,
            'mean_prediction': combination_forecast,
            'Q90': combination_quantile90
        }, index=df_test_norm_diff.index)
    df_weighted_avg['diff_norm_measured'] = df_test_norm_diff['diff_norm_measured']
    return df_weighted_avg

## permutation importance on test set

In [ ]:
def compute_permutation_importances(fitted_model, X_test_augmented, y_test, score_func, quantile, df_train_ensemble_augmented):
    " Compute permutation importances."
    r = permutation_importance(
        fitted_model, X_test_augmented, y_test,
        scoring=score_func[quantile],
        n_repeats=1000,
        random_state=0
    )
    feature_names = df_train_ensemble_augmented.drop(columns=['diff_norm_targ']).columns
    sorted_importances_idx = r['mean_pinball_loss']['importances_mean'].argsort()[::-1]
    importances = pd.DataFrame(
        r['mean_pinball_loss']['importances'][sorted_importances_idx].T,
        columns=feature_names[sorted_importances_idx]
    )
    importances[importances > 0] = 0  # Set positive importances to 0
    abs_importances = importances.abs()  # Get absolute values
    return abs_importances

# plot results

In [ ]:
def plot_elia_forecasts(df, which = 'norm_dayahead'):
    " Plot ELIA forecasts "
    assert which in ['norm_dayahead', 'norm_weekahead', 'norm_dayahead11h'], 'which not in [norm_dayahead, norm_weekahead, norm_dayahead11h]'
    list_cols = [which + 'forecast', which + 'confidence10', which + 'confidence90']
    df[list_cols].plot(color='blue', linestyle='--')
    df['diff_norm_measured'].plot(color='red')

def plot_ensemble_forecasts(df_pred_ensemble, df_ensemble):
    " Plot ensemble forecasts "
    assert 'diff_norm_targ' in list(df_ensemble), 'diff_norm_targ not in df_ensemble'
    # Create a figure and a set of subplots
    fig, ax = plt.subplots()
    # Plot '10_predictions' and '90_predictions' on the same axes with blue dashed lines
    df_plot = df_pred_ensemble[['10_predictions', '90_predictions']]
    df_plot.columns = ['Q10', 'Q90']
    df_plot.plot(ax=ax, color='blue', linestyle='--')
    # Plot '50_predictions' on the same axes with a solid blue line
    df_plot_mean = df_pred_ensemble[['50_predictions']]
    df_plot_mean.columns = ['MEAN']
    df_plot_mean.plot(ax=ax, color='blue')
    df_target = df_ensemble[['diff_norm_targ']]
    df_target.columns = ['target']
    df_target.plot(ax=ax, color='red')

def plot_var_ensemble_forecasts(df_pred_ensemble, df_ensemble):
    " Plot ensemble forecasts "
    assert 'targets' in list(df_ensemble), 'targets not in df_ensemble'
    # Create a figure and a set of subplots
    fig, ax = plt.subplots()
    # Plot '10_predictions' and '90_predictions' on the same axes with blue dashed lines
    df_plot = df_pred_ensemble[['10_var_predictions', '90_var_predictions']]
    df_plot.columns = ['Q10_variability', 'Q90_variability']
    df_plot.plot(ax=ax, color='blue', linestyle='--')
    # Plot '50_predictions' on the same axes with a solid blue line
    df_plot_mean = df_pred_ensemble[['50_var_predictions']]
    df_plot_mean.columns = ['MEAN_variability']
    df_plot_mean.plot(ax=ax, color='blue')
    df_target = df_ensemble[['targets']]
    df_target.columns = ['target_variability']
    df_target.plot(ax=ax, color='red')

def plot_ramp_events(df_test_norm_diff, ABS_DIFFERENCIATE):
    "Plot ramp events"
    if ABS_DIFFERENCIATE:
        plt.axhline(0.3, color='black', linestyle='--')
    else:
        wind_power_changes = df_test_norm_diff['diff_norm_measured'].diff().fillna(0)
        df_test_norm_diff.loc[:, 'ramp'] = (np.abs(wind_power_changes) > 0.3).astype(int)
        # Add vertical lines for ramp events
        ramp_indices = df_test_norm_diff[df_test_norm_diff['ramp'] == 1].index.values
        for idx in ramp_indices:
            plt.axvline(idx, color='k', linestyle='--')

def plot_feature_importance(feature_importance, df_train_ensemble):
    "Plot feature importance"
    assert  len(feature_importance) == len(list(df_train_ensemble)), 'feature_importance and df_train_ensemble have different lengths'
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + 0.5
    _ = plt.figure(figsize=(15, 6))
    plt.subplot(1, 2, 1)
    plt.barh(pos, feature_importance[sorted_idx], align="center")
    plt.yticks(pos, np.array(df_train_ensemble.columns)[sorted_idx])
    plt.title("Feature Importance (training set)")
    plt.show()

def plot_permutation_importances(importances, quantile):
        " Plot permutation importances"
        assert importances is not None, 'importances is None'
        plt.figure(figsize=(15, 6))
        importances.plot.box(vert=False, whis=10)
        plt.title(f"Permutation Importances (test set)")
        plt.axvline(x=0, color="k", linestyle="--")
        if (quantile == 0.1) or (quantile == 0.9):
            plt.xticks(rotation=90)
            plt.xlabel("Decrease in Pinball loss performance")
            return plt.show()
        plt.xticks(rotation=90)
        plt.xlabel("Increase in RMSE loss performance")
        plt.show()

def transform_loss_lists_to_df(lst_loss_gbr_ensemble, lst_loss_equal_weights, lst_loss_weighted_avg, lst_loss_baseline_dayahead, lst_loss_baseline_dayahead11h, lst_loss_baseline_weekahead):
    " Transform the loss lists into a DataFrame"
    assert len(lst_loss_gbr_ensemble) == len(lst_loss_equal_weights) == len(lst_loss_weighted_avg) == len(lst_loss_baseline_dayahead) == len(lst_loss_baseline_dayahead11h) == len(lst_loss_baseline_weekahead), 'Length mismatch'
    # Construct the dictionary from the input lists
    dict_data = {
        'gbr_ensemble': lst_loss_gbr_ensemble,
        'eq_weights': lst_loss_equal_weights,
        'weighted_avg': lst_loss_weighted_avg,
        'dayahead': lst_loss_baseline_dayahead,
        'dayahead11h': lst_loss_baseline_dayahead11h,
        'weekahead': lst_loss_baseline_weekahead,
    }
    # Transform the dictionary into a DataFrame
    data = (
        pd.DataFrame(dict_data)
        .rename_axis('days')  # Set the index name to 'days'
        .melt(                # Melt the DataFrame to long format
            var_name='model',
            value_name='rmse',
            ignore_index=False,
        )
        .reset_index()        # Reset the index to include 'days' as a column
    )
    return data

def plot_statistical_comparison(pc, avg_rank, title1, title2):
    " Plot the statistical comparison"
    # Define the colormap and heatmap arguments
    cmap = ['1', '#fb6a4a',  '#08306b',  '#4292c6', '#c6dbef']
    heatmap_args = {
        'cmap': cmap,
        'linewidths': 0.25,
        'linecolor': '0.5',
        'clip_on': False,
        'square': True,
        'cbar_ax_bbox': [0.80, 0.35, 0.04, 0.3]
    }
    # Plot the heatmap
    plt.title(title1)
    sp.sign_plot(pc, **heatmap_args)
    plt.show()
    # Plot the title for the critical difference diagram
    plt.title(title2)
    # Plot the critical difference diagram
    sp.critical_difference_diagram(avg_rank, pc)
    plt.show()

# run models

In [ ]:
def run_ensemble_predictions_per_quantile(abs_differenciate, X_train, X_test, y_train, df_train_ensemble, 
                                    X_train_quantile10, X_test_quantile10, df_train_ensemble_quantile10, 
                                    X_train_quantile90, X_test_quantile90, df_train_ensemble_quantile90, 
                                    predictions, quantile, iteration, add_quantiles, augment_q50,
                                    nr_cv_splits, model_type, solver, 
                                    gbr_update_every_days, gbr_config_params, lr_config_params,
                                    score_func, PLOT_IMPORT_GBR, best_results, logger):
    " Run ensemble predictions for a specific quantile."
    logger.info('   ')
    logger.opt(colors=True).info(f'<fg 250,128,114> Run ensemble predictions for quantile {quantile} </fg 250,128,114>')
    X_train_augmented, X_test_augmented, df_train_ensemble_augmented = X_train, X_test, df_train_ensemble

    # Augment the training and testing data with the quantiles predictions
    if add_quantiles:
        logger.opt(colors=True).info(f'<fg 250,128,114> Augmenting training and testing data with quantiles </fg 250,128,114>')
        X_train_augmented, X_test_augmented, df_train_ensemble_augmented = augment_with_quantiles(
            X_train, X_test, df_train_ensemble,
            X_train_quantile10, X_test_quantile10, df_train_ensemble_quantile10,
            X_train_quantile90, X_test_quantile90, df_train_ensemble_quantile90,
            quantile, augment_q50=augment_q50)
        
    # Optimize model hyperparameters
    if iteration % gbr_update_every_days == 0:  # Optimize hyperparameters every gbr_update_every_days
        logger.opt(colors=True).info(f'<fg 250,128,114> Optimizing model hyperparameters - updating every {gbr_update_every_days} days</fg 250,128,114>')
        best_score, best_params = optimize_model(X_train_augmented, y_train, quantile,
                                                    nr_cv_splits, model_type, solver, gbr_config_params,
                                                    lr_config_params, score_func, logger)
        best_results[quantile] = [('best_score', best_score), ('params', best_params)]
    else:
        logger.opt(colors=True).info(f'<fg 250,128,114> Using best hyperparameters from first iteration </fg 250,128,114>')
        best_params = best_results[quantile][1][1]
        
    # Initialize model
    model = initialize_model(model_type, quantile, best_params)
    # Fit model
    fitted_model = model.fit(X_train_augmented, y_train)
    if PLOT_IMPORT_GBR and model_type == 'GBR':
        logger.opt(colors=True).info(f'<fg 250,128,114> GBR feature importance </fg 250,128,114>')
        plot_feature_importance(fitted_model.feature_importances_,
                                df_train_ensemble_augmented.drop(columns=['diff_norm_targ']))
    # Make predictions
    raw_predictions = fitted_model.predict(X_test_augmented)
    if not abs_differenciate:
        raw_predictions[raw_predictions < 0] = 0  # Set negative predictions to 0
    predictions[quantile] = raw_predictions  # Store predictions
    return predictions, best_results, fitted_model, X_train_augmented, X_test_augmented, df_train_ensemble_augmented


def run_ensemble_variability_predictions(X_train_2stage, y_train_2stage, X_test_2stage, quantiles, nr_cv_splits, var_model_type, solver, var_gbr_config_params, var_lr_config_params, gbr_update_every_days, score_func, logger, iteration, best_results_var):
    " Run ensemble variability predictions"
    assert var_model_type in ['GBR', 'LR'], 'Invalid model type'
    variability_predictions = {}
    for quantile in tqdm(quantiles, desc='Quantile Regression'):
        logger.opt(colors=True).info(f'<fg 72,201,176> Run ensemble variability predictions for quantile {quantile} </fg 72,201,176>')
        # Optimize model hyperparameters
        if iteration % gbr_update_every_days == 0:  # Optimize hyperparameters every gbr_update_every_days
            logger.opt(colors=True).info(f'<fg 72,201,176> Optimizing model hyperparameters - updating every {gbr_update_every_days} days</fg 72,201,176>')
            best_score, best_params_var = optimize_model(X_train_2stage, y_train_2stage, quantile, nr_cv_splits, var_model_type, solver, var_gbr_config_params, var_lr_config_params, score_func, logger)
            best_results_var[quantile] = [('best_score', best_score), ('params', best_params_var)]
        else:
            logger.opt(colors=True).info(f'<fg 72,201,176> Using best hyperparameters from first iteration </fg 72,201,176>')
            best_params_var = best_results_var[quantile][1][1]
        model = initialize_model(var_model_type, quantile, best_params_var)  # Initialize model
        fitted_model = model.fit(X_train_2stage, y_train_2stage)  # Fit model
        raw_variability_predictions = fitted_model.predict(X_test_2stage)  # Make predictions
        variability_predictions[quantile] = raw_variability_predictions  # Store predictions
    return variability_predictions, best_results_var

## Configuration

In [ ]:
# from dataclasses import dataclass

# @dataclass(frozen=True)
# class Simulation:
#     testing_period = dict( 
#         file_0 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/01.json'
#         file_1 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/02.json'
#         file_2 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/03.json'
#         file_3 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/04.json'
#         file_4 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/05.json'

#         i = 0
#         window_size = 30
#         start_training = '2023-01-05'
#         num_test_days = 80

#         ABS_DIFFERENCIATE = False
#         ADD_QUANTILES = True
#         augment_q50 = False

#         # prediction pipeline
#         max_lags = 3
#         nr_cv_splits = 3
#         quantiles = [0.1, 0.5, 0.9]

#         forecasters_diversity = False
#         lagged = True
#         augment = False
#         differenciate = True

#         baseline_model = 'diff_norm_dayahead'

#         # Ensemble Learning
#         model_type = 'GBR'  # 'GBR' or 'LR'
#         var_model_type = 'GBR'  # 'GBR' or 'LR'

#         gbr_update_every_days = 30

#         # forecasts model parameters
#         gbr_config_params = {'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.101],
#                                 'subsample' : [.8, .85, .95],
#                                 'max_depth': [2, 3, 4, 5],
#                                 'n_estimators': [200, 500]}
#         lr_config_params = {'alpha': [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.7, 0.9, 1],
#                             'fit_intercept' : [True, False]}

#         # variability forecasts model parameters
#         order_diff = 1
#         var_gbr_config_params = {'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.101],
#                                     'subsample' : [.8, .85, .95],
#                                     'max_depth': [2, 3, 4, 5],
#                                     'n_estimators': [200, 500]}
#         var_lr_config_params = {'alpha': [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.7, 0.9, 1],
#                                 'fit_intercept' : [True, False]}

#         SECOND_STAGE = True  # activate the second stage of the ensemble learning
#         PLOT_IMPORT_GBR = False  # plot the feature importances for the GBR model
#         PLOT_IMPORT_PERM = False  # plot the permutation importances
#         ZOOM_VARIABILITY = False # zoom in the variability forecasts
#     )

# sim_params = Simulation.testing_period

In [ ]:
file_0 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/01.json'
file_1 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/02.json'
file_2 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/03.json'
file_3 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/04.json'
file_4 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/05.json'
file_5 = '/Users/gio/Desktop/elia_group/dataset_elia/2023/07.json'

i = 0
window_size = 30
start_training = '2023-01-01'
num_test_days = 90

ABS_DIFFERENCIATE = False
ADD_QUANTILES = True
augment_q50 = True

# prediction pipeline
nr_cv_splits = 5
quantiles = [0.1, 0.5, 0.9]

# params for 1st stage
max_lags = 3
forecasters_diversity = False
lagged = True
augment = True
differenciate = True

# params for 2nd stage
max_lags_var = 3
augment_var=True

baseline_model = 'diff_norm_dayahead'

# Ensemble Learning
model_type = 'GBR'  # 'GBR' or 'LR'
var_model_type = 'GBR'  # 'GBR' or 'LR'

gbr_update_every_days = 30

# forecasts model parameters
gbr_config_params = {'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1],
                        'max_features' : [.8, .85, .95, 1.0],
                        'max_depth': [2, 3, 4],
                        'max_iter': [900, 1000]}
lr_config_params = {'alpha': [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.7, 0.9, 1],
                    'fit_intercept' : [True, False]}

# variability forecasts model parameters
order_diff = 1
var_gbr_config_params = {'learning_rate': [0.001, 0.005, 0.01,  0.05, 0.1],
                            'max_features' : [.8, .85, .95, 1.0],
                            'max_depth': [2, 3, 4],
                            'max_iter': [200, 350, 500]}
var_lr_config_params = {'alpha': [0.005, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.7, 0.9, 1],
                        'fit_intercept' : [True, False]}

SECOND_STAGE = True  # activate the second stage of the ensemble learning
PLOT_IMPORT_GBR = False  # plot the feature importances for the GBR model
PLOT_IMPORT_PERM = False  # plot the permutation importances
ZOOM_VARIABILITY = True # zoom in the variability forecasts


In [ ]:
# consider two months of wind power data
df_01 = process_file(file_0, offshore_filter='Offshore')
df_02 = process_file(file_1, offshore_filter='Offshore')
df_03 = process_file(file_2, offshore_filter='Offshore')
df_04 = process_file(file_3, offshore_filter='Offshore')
df_05 = process_file(file_4, offshore_filter='Offshore')
df_06 = process_file(file_5, offshore_filter='Offshore')
df = pd.concat([df_01, df_02, df_03, df_04, df_05, df_06], axis=0)

# get the maximum capacity
maximum_capacity = df.measured.max()

lst_forecasts = [name for name in list(df.columns) if 'forecast' in name]  # get all forecast columns
lst_confidence10 = [name for name in list(df.columns) if 'confidence10' in name]  # get all confidence10 columns
lst_confidence90 = [name for name in list(df.columns) if 'confidence90' in name]  # get all confidence90 columns

# list of columns to keep
lst_cols = ['measured'] + lst_forecasts + lst_confidence10 + lst_confidence90
df_filtered = df[lst_cols]

In [ ]:
np.random.seed(42)

# loss quantile gradient boosting regressor
lst_rmse_gbr_ensemble = []
# loss equal weights scheme
lst_rmse_equal_weights = []
# loss weighted average scheme
lst_rmse_weighted_avg = []
# loss baseline day ahead
lst_rmse_baseline_dayahead = []
# loss baseline day ahead 11
lst_rmse_baseline_dayahead11h = []
# loss baseline week ahead
lst_rmse_baseline_week_ahead = []


if not ABS_DIFFERENCIATE:
    
    # loss var gradient boosting regressor
    lst_rmse_var_gbr_ensemble = []
    # loss var equal weights scheme
    lst_rmse_var_equal_weights = []
    # loss var weighted average scheme
    lst_rmse_var_weighted_avg = []
    # loss var baseline day ahead
    lst_rmse_var_baseline_dayahead = []
    # loss var baseline day ahead 11
    lst_rmse_var_baseline_dayahead11h = []
    # loss var baseline week ahead
    lst_rmse_var_baseline_week_ahead = []

    # loss quantile gradient boosting regressor
    lst_pb_gbr_ensemble_q10 = []
    lst_pb_gbr_ensemble_q90 = []
    # loss equal weights scheme
    lst_pb_weighted_avg_q10 = []
    lst_pb_weighted_avg_q90 = []
    # loss weighted average scheme
    lst_pb_equal_weights_q10 = []
    lst_pb_equal_weights_q90 = []
    # loss baseline day ahead
    lst_pb_dayahead_q10 = []
    lst_pb_dayahead_q90  = []
    # loss baseline day ahead 11
    lst_pb_dayahead_11h_q10 = []
    lst_pb_dayahead_11h_q90 = []
    # loss baseline week ahead
    lst_pb_week_ahead_q10 = []
    lst_pb_week_ahead_q90 = []

# loop over test days
for i in tqdm(range(num_test_days), desc='Testing Days'):

    # generate timestamps train and prediction
    start_training_timestamp, end_training_timestamp, start_prediction_timestamp, end_prediction_timestamp = generate_timestamps(start_training, i, window_size)

    logger.info(' ')
    logger.opt(colors = True).info('<blue>-------------------------------------------------------------------------------------------</blue>')
    logger.opt(colors = True).info(f'<blue>Start prediction: {start_prediction_timestamp} - End prediction: {end_prediction_timestamp}</blue>')

    df_train = df_filtered[df_filtered.index.to_series().between(start_training_timestamp, end_training_timestamp)].iloc[:-1,:]
    df_test = df_filtered[df_filtered.index.to_series().between(start_prediction_timestamp, end_prediction_timestamp)].iloc[:-1,:]

    logger.opt(colors = True).info('<blue>Forecasters predictions</blue>')
    
    # forecaster - day ahead forecast
    df_day_ahead_pred_train = create_day_ahead_predictions(df_train)
    df_day_ahead_pred_test = create_day_ahead_predictions(df_test)

    # forecaster - day ahead 11 forecast
    df_day_ahead11_pred_train = create_day_ahead_11_predictions(df_train)
    df_day_ahead11_pred_test = create_day_ahead_11_predictions(df_test)

    # forecaster - week ahead forecast
    df_week_ahead_pred_train = create_week_ahead_predictions(df_train)
    df_week_ahead_pred_test = create_week_ahead_predictions(df_test)

    # forecaster - day ahead quantile-10
    df_day_ahead_q10_train = create_day_ahead_quantiles10(df_train)
    df_day_ahead_q10_test = create_day_ahead_quantiles10(df_test)

    # forecaster - day ahead 11 quantile-10
    df_day_ahead11_q10_train = create_day_ahead_11_quantiles10(df_train)
    df_day_ahead11_q10_test = create_day_ahead_11_quantiles10(df_test)

    # forecaster - week ahead quantile-10
    df_week_ahead_q10_train = create_week_ahead_quantiles10(df_train)
    df_week_ahead_q10_test = create_week_ahead_quantiles10(df_test)

    # forecaster - day ahead quantile-90
    df_day_ahead_q90_train = create_day_ahead_quantiles90(df_train)
    df_day_ahead_q90_test = create_day_ahead_quantiles90(df_test)

    # forecaster - day ahead 11 quantile-90
    df_day_ahead11_q90_train = create_day_ahead_11_quantiles90(df_train)
    df_day_ahead11_q90_test = create_day_ahead_11_quantiles90(df_test)

    # forecaster - week ahead quantile-90
    df_week_ahead_q90_train = create_week_ahead_quantiles90(df_train)
    df_week_ahead_q90_test = create_week_ahead_quantiles90(df_test)

    logger.opt(colors = True).info('<blue>Collecting forecasters prediction for ensemble learning </blue>')
    # make esemble dataframe 
    df_train_ensemble = pd.concat([df_day_ahead_pred_train, df_day_ahead11_pred_train, df_week_ahead_pred_train], axis=1) 
    df_test_ensemble = pd.concat([df_day_ahead_pred_test, df_day_ahead11_pred_test, df_week_ahead_pred_test], axis=1)
    df_ensemble = pd.concat([df_train_ensemble, df_test_ensemble], axis=0)

    if ADD_QUANTILES:
        df_train_ensemble_quantile10 = pd.concat([df_day_ahead_q10_train, df_day_ahead11_q10_train, df_week_ahead_q10_train], axis=1)
        df_train_ensemble_quantile90 = pd.concat([df_day_ahead_q90_train, df_day_ahead11_q90_train, df_week_ahead_q90_train], axis=1)
        df_test_ensemble_quantile10 = pd.concat([df_day_ahead_q10_test, df_day_ahead11_q10_test, df_week_ahead_q10_test], axis=1)
        df_test_ensemble_quantile90 = pd.concat([df_day_ahead_q90_test, df_day_ahead11_q90_test, df_week_ahead_q90_test], axis=1)
        df_ensemble_quantile10 = pd.concat([df_train_ensemble_quantile10, df_test_ensemble_quantile10], axis=0)
        df_ensemble_quantile90 = pd.concat([df_train_ensemble_quantile90, df_test_ensemble_quantile90], axis=0)

    df_ensemble_normalized = normalize_dataframe(df_ensemble, maximum_capacity)
    if ADD_QUANTILES:
        df_ensemble_normalized_quantile10 = normalize_dataframe(df_ensemble_quantile10, maximum_capacity)
        df_ensemble_normalized_quantile90 = normalize_dataframe(df_ensemble_quantile90, maximum_capacity)

    df_ensemble_normalized_lag = create_augmented_dataframe(df_ensemble_normalized, max_lags=max_lags, forecasters_diversity=forecasters_diversity, lagged=lagged, augmented=augment, differenciate=differenciate)
    if ADD_QUANTILES:
        df_ensemble_normalized_lag_quantile10 = create_augmented_dataframe(df_ensemble_normalized_quantile10, max_lags=max_lags, forecasters_diversity=forecasters_diversity, lagged=lagged, augmented=augment, differenciate=differenciate)
        df_ensemble_normalized_lag_quantile90 = create_augmented_dataframe(df_ensemble_normalized_quantile90, max_lags=max_lags, forecasters_diversity=forecasters_diversity, lagged=lagged, augmented=augment, differenciate=differenciate)

    # concatenate train and test dataframes
    df_process = pd.concat([df_train, df_test], axis=0)

    # normalize dataframe
    df_process_norm = normalize_dataframe(df_process, maximum_capacity)

    # differenciate dataframe
    if ABS_DIFFERENCIATE:
        df_process_norm_diff = differentiate_dataframe(df_process_norm)
    else:
        df_process_norm_diff = df_process_norm.copy()
        lst_cols_diff = ['diff_' + name for name in list(df_process_norm.columns)]
        df_process_norm_diff.columns = lst_cols_diff

    df_train_norm_diff, df_test_norm_diff = df_process_norm_diff[df_process_norm_diff.index < start_prediction_timestamp], df_process_norm_diff[df_process_norm_diff.index >= start_prediction_timestamp]
    df_train_ensemble, df_test_ensemble = prepare_train_test_data(df_ensemble_normalized_lag, df_train_norm_diff, df_test_norm_diff, start_prediction_timestamp)
    if ADD_QUANTILES:
        df_train_ensemble_quantile10, df_test_ensemble_quantile10 = df_ensemble_normalized_lag_quantile10[df_ensemble_normalized_lag_quantile10.index< start_prediction_timestamp], df_ensemble_normalized_lag_quantile10[df_ensemble_normalized_lag_quantile10.index>= start_prediction_timestamp]
        df_train_ensemble_quantile90, df_test_ensemble_quantile90 = df_ensemble_normalized_lag_quantile90[df_ensemble_normalized_lag_quantile90.index< start_prediction_timestamp], df_ensemble_normalized_lag_quantile90[df_ensemble_normalized_lag_quantile90.index>= start_prediction_timestamp]

    # assert df_test match df_ensemble_test
    assert (df_test_norm_diff['diff_norm_measured'] == df_test_ensemble['diff_norm_targ']).all()

    # make X-y train and test sets
    X_train, y_train, X_test, y_test = get_numpy_Xy_train_test(df_train_ensemble, df_test_ensemble)

    if ADD_QUANTILES:
        X_train_quantile10, X_test_quantile10 = df_train_ensemble_quantile10.values, df_test_ensemble_quantile10.values
        X_train_quantile90, X_test_quantile90 = df_train_ensemble_quantile90.values, df_test_ensemble_quantile90.values

    # assert do not have nans (should do it before in processing file)
    assert df_train_ensemble.isna().sum().sum() == 0

    # run ensemble learning
    logger.info('   ')
    logger.opt(colors=True).info(f'<fg 250,128,114> Compute Predictions </fg 250,128,114>')
    predictions = {}
    if i==0:
        best_results = {}
    for quantile in tqdm(quantiles, desc='Quantile Regression'):

        # run ensemble predictions
        predictions, best_results, fitted_model, X_train_augmented, X_test_augmented, df_train_ensemble_augmented = run_ensemble_predictions_per_quantile(abs_differenciate=ABS_DIFFERENCIATE,X_train=X_train, X_test=X_test, y_train=y_train, df_train_ensemble=df_train_ensemble, 
                                    X_train_quantile10=X_train_quantile10, X_test_quantile10=X_test_quantile10, df_train_ensemble_quantile10=df_train_ensemble_quantile10, 
                                    X_train_quantile90=X_train_quantile90, X_test_quantile90=X_test_quantile90, df_train_ensemble_quantile90=df_train_ensemble_quantile90, 
                                    predictions=predictions, quantile=quantile, add_quantiles=ADD_QUANTILES, augment_q50=augment_q50,
                                    nr_cv_splits=nr_cv_splits, model_type=model_type, solver=solver, 
                                    gbr_update_every_days=gbr_update_every_days, gbr_config_params=gbr_config_params, lr_config_params=lr_config_params,
                                    score_func=score_func, PLOT_IMPORT_GBR=PLOT_IMPORT_GBR, best_results=best_results, iteration=i, logger=logger)

        if PLOT_IMPORT_PERM:
            logger.info('   ')
            logger.opt(colors=True).info(f'<fg 250,128,114> EX-Post Payments with loss-based importance </fg 250,128,114>')
            abs_importances = compute_permutation_importances(fitted_model, X_test_augmented, y_test, score_func, quantile, df_train_ensemble_augmented)
            plot_permutation_importances(abs_importances, quantile)
            logger.info('   ')

        if not ABS_DIFFERENCIATE:
            if SECOND_STAGE and quantile == 0.5:
                logger.info('   ')
                logger.opt(colors=True).info(f'<fg 72,201,176> Compute Variability Predictions </fg 72,201,176>')
                # make predictions for variability
                predictions_insample = fitted_model.predict(X_train_augmented)
                predictions_outsample = fitted_model.predict(X_test_augmented)
                # create 2-stage dataframe
                df_2stage = create_2stage_dataframe(df_train_ensemble, df_test_ensemble, y_train, y_test, predictions_insample, predictions_outsample)
                # differenciate dataframe with lags
                df_2stage_process = create_augmented_dataframe_2stage(df_2stage, order_diff, max_lags=max_lags_var, augment=augment_var)
                # split train and test
                df_2stage_train, df_2stage_test = df_2stage_process[df_2stage_process.index < start_prediction_timestamp], df_2stage_process[df_2stage_process.index >= start_prediction_timestamp]
                # make X-y train and test sets for 2-stage model
                X_train_2stage, y_train_2stage, X_test_2stage, y_test_2stage = df_2stage_train.drop(columns=['targets']).values, df_2stage_train['targets'].values, df_2stage_test.drop(columns=['targets']).values, df_2stage_test['targets'].values
                # run ensemble learning
                if i == 0:
                    best_results_var = {}
                # run variability predictions
                variability_predictions, best_results_var = run_ensemble_variability_predictions(X_train_2stage = X_train_2stage, y_train_2stage=y_train_2stage, X_test_2stage=X_test_2stage, 
                                                                                                    quantiles=quantiles, nr_cv_splits=nr_cv_splits, var_model_type=var_model_type, solver=solver, 
                                                                                                    var_gbr_config_params=var_gbr_config_params, var_lr_config_params=var_lr_config_params, gbr_update_every_days=gbr_update_every_days, 
                                                                                                    score_func=score_func, logger=logger, iteration=i, best_results_var=best_results_var)
                # collect results as dictionary
                var_predictions_dict = collect_quantile_ensemble_predictions(quantiles, df_2stage_test, variability_predictions)
                df_var_ensemble = create_var_ensemble_dataframe(quantiles, var_predictions_dict, df_2stage_test)

        logger.info('   ')
        del X_train_augmented, X_test_augmented, df_train_ensemble_augmented
        gc.collect()

    # collect results as dictionary
    quantile_predictions_dict = collect_quantile_ensemble_predictions(quantiles, df_test_norm_diff, predictions)
    
    # collect results as dataframe
    df_pred_ensemble = create_ensemble_dataframe(quantiles, quantile_predictions_dict, df_test_norm_diff)

    # performance ensemble
    rmse_ensemble = round(calculate_rmse(df_pred_ensemble, '50_predictions').values[0][0], 3)
    lst_rmse_gbr_ensemble.append(rmse_ensemble)
    if not ABS_DIFFERENCIATE:
        pinball_ensemble = calculate_pinball_losses(df_pred_ensemble, '10_predictions', '90_predictions')
        pinball_ensemble_q10 = round(pinball_ensemble['pb_loss_10'].values[0], 3)
        pinball_ensemble_q90 = round(pinball_ensemble['pb_loss_90'].values[0], 3)
        lst_pb_gbr_ensemble_q10.append(pinball_ensemble_q10)
        lst_pb_gbr_ensemble_q90.append(pinball_ensemble_q90)

    if not ABS_DIFFERENCIATE:

        # performance variability ensemble
        rmse_var_ensemble = round(calculate_rmse(df_var_ensemble, '50_var_predictions', targ_col='targets').values[0][0], 3)
        lst_rmse_var_gbr_ensemble.append(rmse_var_ensemble)

        df_weighted_avg_var = calculate_weighted_avg(df_train_norm_diff, df_test_norm_diff, start_prediction_timestamp, window_size_valid=1, var=True)
        rmse_var_weighted_avg = round(calculate_rmse(df_weighted_avg_var, 'mean_prediction').values[0][0], 3)
        lst_rmse_var_weighted_avg.append(rmse_var_weighted_avg)

        # concatenate last training row with test data
        df_test_norm_var = pd.concat([df_train_norm_diff.iloc[-1:, :], df_test_norm_diff], axis=0).diff().iloc[1:, :]

        df_equal_weights_var = calculate_equal_weights(df_test_norm_var)
        rmse_var_equal_weights = round(calculate_rmse(df_equal_weights_var, 'mean_prediction').values[0][0], 3)
        lst_rmse_var_equal_weights.append(rmse_var_equal_weights)

        df_dayahead_var = df_test_norm_var[['diff_norm_dayaheadforecast', 'diff_norm_measured']]
        rmse_var_dayahead = round(calculate_rmse(df_dayahead_var, 'diff_norm_dayaheadforecast').values[0][0], 3)
        lst_rmse_var_baseline_dayahead.append(rmse_var_dayahead)

        df_dayahead_11h_var = df_test_norm_var[['diff_norm_dayahead11hforecast', 'diff_norm_measured']]
        rmse_var_dayahead_11h = round(calculate_rmse(df_dayahead_11h_var, 'diff_norm_dayahead11hforecast').values[0][0], 3)
        lst_rmse_var_baseline_dayahead11h.append(rmse_var_dayahead_11h)

        # performance week ahead
        df_week_ahead_var = df_test_norm_var[['diff_norm_weekaheadforecast', 'diff_norm_measured']]
        rmse_var_week_ahead = round(calculate_rmse(df_week_ahead_var, 'diff_norm_weekaheadforecast').values[0][0], 3)
        lst_rmse_var_baseline_week_ahead.append(rmse_var_week_ahead)

    # performance weighted average
    df_weighted_avg = calculate_weighted_avg(df_train_norm_diff, df_test_norm_diff, start_prediction_timestamp, window_size_valid=1)
    rmse_weighted_avg = round(calculate_rmse(df_weighted_avg, 'mean_prediction').values[0][0], 3)
    lst_rmse_weighted_avg.append(rmse_weighted_avg)
    if not ABS_DIFFERENCIATE:
        pinball_weighted_avg = calculate_pinball_losses(df_weighted_avg, 'Q10', 'Q90')
        pinball_weighted_avg_q10 = round(pinball_weighted_avg['pb_loss_10'].values[0], 3)
        pinball_weighted_avg_q90 = round(pinball_weighted_avg['pb_loss_90'].values[0], 3)
        lst_pb_weighted_avg_q10.append(pinball_weighted_avg_q10)
        lst_pb_weighted_avg_q90.append(pinball_weighted_avg_q90)

    # performance equal weights
    df_equal_weights = calculate_equal_weights(df_test_norm_diff)
    rmse_equal_weights = round(calculate_rmse(df_equal_weights, 'mean_prediction').values[0][0], 3)
    lst_rmse_equal_weights.append(rmse_equal_weights)
    if not ABS_DIFFERENCIATE:
        pinball_equal_weights = calculate_pinball_losses(df_equal_weights, 'Q10', 'Q90')
        pinball_equal_weights_q10 = round(pinball_equal_weights['pb_loss_10'].values[0], 3)
        pinball_equal_weights_q90 = round(pinball_equal_weights['pb_loss_90'].values[0], 3)
        lst_pb_equal_weights_q10.append(pinball_equal_weights_q10)
        lst_pb_equal_weights_q90.append(pinball_equal_weights_q90)

    # performance day-ahead
    df_dayahead = df_test_norm_diff[['diff_norm_dayaheadforecast', 'diff_norm_dayaheadconfidence10', 'diff_norm_dayaheadconfidence90', 'diff_norm_measured']]
    rmse_dayahead = round(calculate_rmse(df_dayahead, 'diff_norm_dayaheadforecast').values[0][0], 3)
    lst_rmse_baseline_dayahead.append(rmse_dayahead)
    if not ABS_DIFFERENCIATE:
        pinball_dayahead = calculate_pinball_losses(df_dayahead, 'diff_norm_dayaheadconfidence10', 'diff_norm_dayaheadconfidence90')
        pinball_dayahead_q10 = round(pinball_dayahead['pb_loss_10'].values[0], 3)
        pinball_dayahead_q90 = round(pinball_dayahead['pb_loss_90'].values[0], 3)
        lst_pb_dayahead_q10.append(pinball_dayahead_q10)
        lst_pb_dayahead_q90.append(pinball_dayahead_q90)
    
    # performance day-ahead-11h
    df_dayahead_11h = df_test_norm_diff[['diff_norm_dayahead11hforecast', 'diff_norm_dayahead11hconfidence10', 'diff_norm_dayahead11hconfidence90', 'diff_norm_measured']]
    rmse_dayahead_11h = round(calculate_rmse(df_dayahead_11h, 'diff_norm_dayahead11hforecast').values[0][0], 3)
    lst_rmse_baseline_dayahead11h.append(rmse_dayahead_11h)
    if not ABS_DIFFERENCIATE:
        pinball_dayahead_11h = calculate_pinball_losses(df_dayahead_11h, 'diff_norm_dayahead11hconfidence10', 'diff_norm_dayahead11hconfidence90')
        pinball_dayahead_11h_q10 = round(pinball_dayahead_11h['pb_loss_10'].values[0], 3)
        pinball_dayahead_11h_q90 = round(pinball_dayahead_11h['pb_loss_90'].values[0], 3)
        lst_pb_dayahead_11h_q10.append(pinball_dayahead_11h_q10)
        lst_pb_dayahead_11h_q90.append(pinball_dayahead_11h_q90)

    # performance week ahead
    df_week_ahead = df_test_norm_diff[['diff_norm_weekaheadforecast', 'diff_norm_weekaheadconfidence10', 'diff_norm_weekaheadconfidence90', 'diff_norm_measured']]
    rmse_week_ahead = round(calculate_rmse(df_week_ahead, 'diff_norm_weekaheadforecast').values[0][0], 3)
    lst_rmse_baseline_week_ahead.append(rmse_week_ahead)
    if not ABS_DIFFERENCIATE:
        pinball_week_ahead = calculate_pinball_losses(df_week_ahead, 'diff_norm_weekaheadconfidence10', 'diff_norm_weekaheadconfidence90')
        pinball_week_ahead_q10 = round(pinball_week_ahead['pb_loss_10'].values[0], 3)
        pinball_week_ahead_q90 = round(pinball_week_ahead['pb_loss_90'].values[0], 3)
        lst_pb_week_ahead_q10.append(pinball_week_ahead_q10)
        lst_pb_week_ahead_q90.append(pinball_week_ahead_q90)

    # plot forecasts
    plot_ensemble_forecasts(df_pred_ensemble, df_test_ensemble)
    nr_previous_days = len(pd.date_range(start=start_training_timestamp, end=end_training_timestamp, freq='1D')) - 1
    plt.title(f'Ensemble Forecasts - Quantile {model_type}')
    plot_ramp_events(df_test_norm_diff, ABS_DIFFERENCIATE)
    if not ABS_DIFFERENCIATE:
        plt.ylim(-0.01, 1)
    plt.show()

    if not ABS_DIFFERENCIATE:
        # plot variability forecast results
        plot_var_ensemble_forecasts(df_var_ensemble, df_2stage_test)
        nr_previous_days = len(pd.date_range(start=start_training_timestamp, end=end_training_timestamp, freq='1D')) - 1
        plt.title(f'Ensemble Variability Forecasts - Quantile {model_type}')
        plot_ramp_events(df_test_norm_diff, ABS_DIFFERENCIATE)
        if not ZOOM_VARIABILITY:
            plt.ylim(-0.6, 0.6)
        plt.show()

    logger.info(' ')
    logger.info('------------- Wind Power Froecasting ------------------------------')
    logger.info(f'----------------- RMSE -----------------')
    logger.info(f'GBR Stacked {rmse_ensemble}')
    logger.info(f'Weighted Average {rmse_weighted_avg}')
    logger.info(f'Equal Weights {rmse_equal_weights}')
    logger.info(f'Day-Ahead {rmse_dayahead}')
    logger.info(f'Day-Ahead-11h {rmse_dayahead_11h}')
    logger.info(f'Week-Ahead {rmse_week_ahead}')
    logger.info('   ')
    
    if not ABS_DIFFERENCIATE:
        logger.info(f'----------------- PB Q10 -----------------')
        logger.info(f'PB Q10 GBR Stacked {pinball_ensemble_q10}')
        logger.info(f'PB Q10 Weig Avg {pinball_weighted_avg_q10}')
        logger.info(f'PB Q10 Eq Weig {pinball_weighted_avg_q10}')
        logger.info(f'PB Q10 Day-Ahead {pinball_dayahead_q10}')
        logger.info(f'PB Q10 Day-Ahead-11h {pinball_dayahead_11h_q10}')
        logger.info(f'PB Q10 Week-Ahead {pinball_week_ahead_q10}')
        logger.info('   ')
        logger.info(f'----------------- PB Q90 -----------------')
        logger.info(f'GBR Stacked {pinball_ensemble_q90}')
        logger.info(f'Weig Avg {pinball_weighted_avg_q90}')
        logger.info(f'Eq Weig {pinball_weighted_avg_q90}')
        logger.info(f'Day-Ahead {pinball_dayahead_q90}')
        logger.info(f'Day-Ahead-11h {pinball_dayahead_11h_q90}')
        logger.info(f'Week-Ahead {pinball_week_ahead_q90}')
        logger.info('   ')
        logger.info(f'----------------- Wind Power Variability Forecast -----------------')
        logger.info(f'----------------- RMSE -----------------')
        logger.info(f'GBR Stacked {rmse_var_ensemble}')
        logger.info(f'Weighted Average {rmse_var_weighted_avg}')
        logger.info(f'Equal Weights {rmse_var_equal_weights}')
        logger.info(f'Day-Ahead {rmse_var_dayahead}')
        logger.info(f'Day-Ahead-11h {rmse_var_dayahead_11h}')
        logger.info(f'Week-Ahead {rmse_var_week_ahead}')
        logger.info('   ')

In [ ]:
data = transform_loss_lists_to_df(lst_rmse_gbr_ensemble, lst_rmse_equal_weights, lst_rmse_weighted_avg, lst_rmse_baseline_dayahead, lst_rmse_baseline_dayahead11h, lst_rmse_baseline_week_ahead)
avg_rank = data.groupby('days').rmse.rank(pct=True).groupby(data.model).mean()
pc = sp.posthoc_nemenyi_friedman(data, y_col='rmse', block_col='days', group_col='model', melted=True)
plot_statistical_comparison(pc, avg_rank,
                            title1 = 'RMSE-based Statistical Significance',
                            title2 = 'RMSE-based Statistical Comparison: critical difference diagram of ranks')

if not ABS_DIFFERENCIATE:

    data = transform_loss_lists_to_df(lst_pb_gbr_ensemble_q10, lst_pb_equal_weights_q10, lst_pb_weighted_avg_q10, lst_pb_dayahead_q10, lst_pb_dayahead_11h_q10, lst_pb_week_ahead_q10)
    avg_rank = data.groupby('days').rmse.rank(pct=True).groupby(data.model).mean()
    pc = sp.posthoc_nemenyi_friedman(data, y_col='rmse', block_col='days', group_col='model', melted=True)
    plot_statistical_comparison(pc, avg_rank,
                                title1 = 'Q10 Pinball loss-based Statistical Significance',
                                title2 = 'Q10 Pinball loss-based Statistical Comparison: critical difference diagram of ranks')
    
    data = transform_loss_lists_to_df(lst_pb_gbr_ensemble_q90, lst_pb_equal_weights_q90, lst_pb_weighted_avg_q90, lst_pb_dayahead_q90, lst_pb_dayahead_11h_q90, lst_pb_dayahead_11h_q90)
    avg_rank = data.groupby('days').rmse.rank(pct=True).groupby(data.model).mean()
    pc = sp.posthoc_nemenyi_friedman(data, y_col='rmse', block_col='days', group_col='model', melted=True)
    plot_statistical_comparison(pc, avg_rank,
                                title1 = 'Q90 Pinball loss-based Statistical Significance',
                                title2 = 'Q90 Pinball loss-based Statistical Comparison: critical difference diagram of ranks')
    
    data = transform_loss_lists_to_df(lst_rmse_var_gbr_ensemble, lst_rmse_var_equal_weights, lst_rmse_var_weighted_avg, lst_rmse_var_baseline_dayahead, lst_rmse_var_baseline_dayahead11h, lst_rmse_var_baseline_week_ahead)
    avg_rank = data.groupby('days').rmse.rank(pct=True).groupby(data.model).mean()
    pc = sp.posthoc_nemenyi_friedman(data, y_col='rmse', block_col='days', group_col='model', melted=True)
    plot_statistical_comparison(pc, avg_rank, 
                                title1 = 'RMSE-based Statistical Significance',
                                title2 = 'RMSE-based Statistical Comparison: critical difference diagram of ranks')